In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

In [2]:
from pyomo.environ import (
    Constraint,
    Var,
    ConcreteModel,
    Expression,
    Objective,
    TransformationFactory,
    value,
    units as pyunits,
)
from pyomo.network import Arc

from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock,
)
from idaes.models.properties.modular_properties.base.generic_reaction import (
    GenericReactionParameterBlock,
)
from idaes.models.unit_models import Feed, Mixer, Heater, StoichiometricReactor, Product

from idaes.core.solvers import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state

In [3]:
import egprod_ideal as thermo_props
import egprod_reaction as reaction_props

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

In [5]:
m.fs.thermo_params = GenericParameterBlock(**thermo_props.config_dict)
m.fs.reaction_params = GenericReactionParameterBlock(
    property_package=m.fs.thermo_params, **reaction_props.config_dict
)

In [6]:
m.fs.OXIDE = Feed(property_package=m.fs.thermo_params)
m.fs.ACID = Feed(property_package=m.fs.thermo_params)
m.fs.PROD = Product(property_package=m.fs.thermo_params)
m.fs.M101 = Mixer(
    property_package=m.fs.thermo_params, inlet_list=["reagent_feed", "catalyst_feed"]
)
m.fs.H101 = Heater(
    property_package=m.fs.thermo_params,
    has_pressure_change=False,
    has_phase_equilibrium=False,
)

In [7]:
m.fs.R101 = StoichiometricReactor(
    property_package=m.fs.thermo_params,
    reaction_package=m.fs.reaction_params,
    has_heat_of_reaction=True,
    has_heat_transfer=True,
    has_pressure_change=False,
)

In [8]:
m.fs.s01 = Arc(source=m.fs.OXIDE.outlet, destination=m.fs.M101.reagent_feed)
m.fs.s02 = Arc(source=m.fs.ACID.outlet, destination=m.fs.M101.catalyst_feed)
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.PROD.inlet)

In [9]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [10]:
m.fs.eg_prod = Expression(
    expr=pyunits.convert(
        m.fs.PROD.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]
        * m.fs.thermo_params.ethylene_glycol.mw,  # MW defined in properties as kg/mol
        to_units=pyunits.Mlb / pyunits.yr,
    )
)  # converting kg/s to MM lb/year

In [11]:
m.fs.cooling_cost = Expression(
    expr=2.12e-8 * (-m.fs.R101.heat_duty[0])
)  # the reaction is exothermic, so R101 duty is negative
m.fs.heating_cost = Expression(
    expr=2.2e-7 * m.fs.H101.heat_duty[0]
)  # the stream must be heated to T_rxn, so H101 duty is positive
m.fs.operating_cost = Expression(
    expr=(3600 * 8000 * (m.fs.heating_cost + m.fs.cooling_cost))
)

In [12]:
print(degrees_of_freedom(m))

15


In [13]:
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(
    58.0 * pyunits.mol / pyunits.s
)
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "water"].fix(
    39.6 * pyunits.mol / pyunits.s
)  # calculated from 16.1 mol EO / cudm in stream
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.OXIDE.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.OXIDE.outlet.temperature.fix(298.15 * pyunits.K)
m.fs.OXIDE.outlet.pressure.fix(1e5 * pyunits.Pa)

m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "water"].fix(
    200 * pyunits.mol / pyunits.s
)
m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(
    0.334 * pyunits.mol / pyunits.s
)  # calculated from 0.9 wt% SA in stream
m.fs.ACID.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(
    1e-5 * pyunits.mol / pyunits.s
)
m.fs.ACID.outlet.temperature.fix(298.15 * pyunits.K)
m.fs.ACID.outlet.pressure.fix(1e5 * pyunits.Pa)

In [14]:
m.fs.H101.outlet.temperature.fix(328.15 * pyunits.K)

In [15]:
m.fs.R101.conversion = Var(
    initialize=0.80, bounds=(0, 1), units=pyunits.dimensionless
)  # fraction

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion
    * m.fs.R101.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"]
    == (
        m.fs.R101.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"]
        - m.fs.R101.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"]
    )
)

m.fs.R101.conversion.fix(0.80)

m.fs.R101.outlet.temperature.fix(328.15 * pyunits.K)  # equal inlet reactor temperature

In [16]:
print(degrees_of_freedom(m))

0


In [17]:
# Initialize and solve each unit operation
m.fs.OXIDE.initialize()
propagate_state(arc=m.fs.s01)

m.fs.ACID.initialize()
propagate_state(arc=m.fs.s01)

m.fs.M101.initialize()
propagate_state(arc=m.fs.s03)

m.fs.H101.initialize()
propagate_state(arc=m.fs.s04)

m.fs.R101.initialize()
propagate_state(arc=m.fs.s05)

m.fs.PROD.initialize()

# set solver
solver = get_solver()

2023-02-18 12:05:09 [INFO] idaes.init.fs.OXIDE.properties: Starting initialization


2023-02-18 12:05:09 [INFO] idaes.init.fs.OXIDE.properties: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:09 [INFO] idaes.init.fs.OXIDE.properties: Property package initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:09 [INFO] idaes.init.fs.OXIDE: Initialization Complete.


2023-02-18 12:05:09 [INFO] idaes.init.fs.ACID.properties: Starting initialization


...

2023-02-18 12:05:11 [INFO] idaes.init.fs.R101: Initialization Complete: optimal - Optimal Solution Found


2023-02-18 12:05:11 [INFO] idaes.init.fs.PROD.properties: Starting initialization


2023-02-18 12:05:11 [INFO] idaes.init.fs.PROD.properties: Property initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:11 [INFO] idaes.init.fs.PROD.properties: Property package initialization: optimal - Optimal Solution Found.


2023-02-18 12:05:11 [INFO] idaes.init.fs.PROD: Initialization Complete.


In [18]:
# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [19]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

operating cost = $3.458 million per year


In [20]:
m.fs.R101.report()

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")


Unit : fs.R101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                  : Value       : Units         : Fixed : Bounds
               Heat Duty : -5.6566e+06 :          watt : False : (None, None)
    Reaction Extent [R1] :      46.400 : mole / second : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                  Units         Inlet     Outlet  
    Molar Flowrate ('Liq', 'ethylene_oxide')   mole / second     58.000     11.600
    Molar Flowrate ('Liq', 'water')            mole / second     239.60     193.20
    Molar Flowrate ('Liq', 'sulfuric_acid')    mole / second    0.33401    0.33401
    Molar Flowrate ('Liq', 'ethylene_glycol')  mole / second 2.0000e-05     46.400
    Temperature                             

In [21]:
m.fs.objective = Objective(expr=m.fs.operating_cost)

In [22]:
m.fs.eg_prod_con = Constraint(
    expr=m.fs.eg_prod >= 200 * pyunits.Mlb / pyunits.yr
)  # MM lb/year
m.fs.R101.conversion.fix(0.90)

m.fs.H101.outlet.temperature.unfix()
m.fs.H101.outlet.temperature[0].setlb(328.15 * pyunits.K)
m.fs.H101.outlet.temperature[0].setub(
    470.45 * pyunits.K
)  # highest component boiling point (ethylene glycol)

m.fs.R101.outlet.temperature.unfix()

In [23]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [24]:
print(f"operating cost = ${value(m.fs.operating_cost)/1e6:0.3f} million per year")

print()
print("Heater results")

m.fs.H101.report()

print()
print("Stoichiometric reactor results")

m.fs.R101.report()

operating cost = $3.888 million per year

Heater results

Unit : fs.H101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value  : Units : Fixed : Bounds
    Heat Duty : 699.26 :  watt : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                  Units         Inlet     Outlet  
    Molar Flowrate ('Liq', 'ethylene_oxide')   mole / second     58.000     58.000
    Molar Flowrate ('Liq', 'water')            mole / second     239.60     239.60
    Molar Flowrate ('Liq', 'sulfuric_acid')    mole / second    0.33401    0.33401
    Molar Flowrate ('Liq', 'ethylene_glycol')  mole / second 2.0000e-05 2.0000e-05
    Temperature                                       kelvin     298.15     328.15
    Pressure                  

In [25]:
print("Optimal Values")
print()

print(f"H101 outlet temperature = {value(m.fs.H101.outlet.temperature[0]):0.3f} K")

print()
print(f"R101 outlet temperature = {value(m.fs.R101.outlet.temperature[0]):0.3f} K")

print()
print(f"Ethylene glycol produced = {value(m.fs.eg_prod):0.3f} MM lb/year")

print()
print(f"Conversion achieved = {value(m.fs.R101.conversion):.1%}")

Optimal Values

H101 outlet temperature = 328.150 K

R101 outlet temperature = 450.000 K

Ethylene glycol produced = 225.415 MM lb/year

Conversion achieved = 90.0%
